# Chains in LangChain（LangChain中的链）

## Outline

* LLMChain（大语言模型链）
* Sequential Chains（顺序链）
  * SimpleSequentialChain
  * SequentialChain
* Router Chain（路由链）

### 为什么我们需要Chains ？
链允许我们将多个组件组合在一起，以创建一个单一的、连贯的应用程序。链（Chains）通常将一个LLM（大语言模型）与提示结合在一起，使用这个构建块，您还可以将一堆这些构建块组合在一起，对您的文本或其他数据进行一系列操作。例如，我们可以创建一个链，该链接受用户输入，使用提示模板对其进行格式化，然后将格式化的响应传递给LLM。我们可以通过将多个链组合在一起，或者通过将链与其他组件组合在一起来构建更复杂的链。

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:

import sys
sys.path.append("D:\\software\\conda\\envs\\torch2\\Lib\\site-packages")

In [2]:
!pip install pandas

  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.8 MB/s  0:00:02m 6.8 MB/s eta 0:00:01
Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]━━━━ 3/4 [pandas]


这些链的一部分的强大之处在于你可以一次运行它们在许多输入上，因此，我们将加载一个pandas数据框架

In [4]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [5]:
df.head()

,Product,Review
0,cea,1212


## 1. LLMChain

LLMChain是一个简单但非常强大的链，也是后面我们将要介绍的许多链的基础。

In [6]:
# from langchain.chat_models import ChatOpenAI    #导入OpenAI模型
# from langchain.prompts import ChatPromptTemplate   #导入聊天提示模板
# from langchain.chains import LLMChain    #导入LLM链。
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import os

初始化语言模型

In [ ]:
# 初始化组件
# 移除或禁用可能会导致验证错误的不受支持代理（例如 socks://）
for proxy_var in ("ALL_PROXY", "all_proxy", "HTTP_PROXY", "http_proxy", "HTTPS_PROXY", "https_proxy"):
    os.environ.pop(proxy_var, None)

llm  = ChatOpenAI(
    temperature=0.7,
    model="glm-4-air-0111",
    openai_api_key="3f2d375ac7e109114bf9f18b3bee1c70.i0MF925p6KNu0KS1",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

初始化prompt，这个prompt将接受一个名为product的变量。该prompt将要求LLM生成一个描述制造该产品的公司的最佳名称

In [21]:
prompt = ChatPromptTemplate.from_template(   
    "What is the best name to describe \
    a company that makes {product}?"
)

将llm和prompt组合成链---这个LLM链非常简单，他只是llm和prompt的结合，但是现在，这个链让我们可以以一种顺序的方式去通过prompt运行并且结合到LLM中

In [22]:
# chain = LLMChain(llm=llm, prompt=prompt)

# 进阶:使用LCEL(LangChain Expression Language)链式调用
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt
    | llm
    
)

因此，如果我们有一个名为"Queen Size Sheet Set"的产品，我们可以通过使用chain.run将其通过这个链运行

In [23]:
product = "Queen Size Sheet Set"
chain.invoke(product)

BadRequestError: Error code: 400 - {'error': {'code': '1211', 'message': '模型不存在，请检查模型代码。'}}

您可以输入任何产品描述，然后查看链将输出什么结果

In [12]:
# 中文
prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的公司的最佳名称是什么?"
)
chain = (llm | prompt)
product = "蓝牙耳机"
chain.invoke(product)

ChatPromptValue(messages=[HumanMessage(content="描述制造content='\\n好的，我们来全面地聊一聊“蓝牙耳机”。\\n\\n这已经是一个非常普遍的电子产品，但选择起来其实门道不少。我会从**类型、选购要点、主流品牌、常见问题**等几个方面，为你提供一个完整的指南。\\n\\n---\\n\\n### 一、蓝牙耳机的主要类型\\n\\n选择蓝牙耳机，首先要确定你想要哪种形态。目前主流的有三种：\\n\\n1.  **真无线耳机**\\n    *   **形态**：左右两个耳机完全独立，没有线缆连接，通过一个充电盒收纳和充电。\\n    *   **优点**：最方便、最自由，无任何束缚。是目前市场的主流。\\n    *   **缺点**：小巧，容易丢失；续航依赖充电盒。\\n    *   **适合场景**：日常通勤、办公、运动、几乎所有场景。\\n\\n2.  **头戴式蓝牙耳机**\\n    *   **形态**：传统的“大耳机”形态，耳罩完全包裹或压在耳朵上。\\n    *   **优点**：音质通常最好（物理空间大，能放更大的单元）；降噪效果顶级（物理隔音+主动降噪）；续航时间长。\\n    *   **缺点**：体积大，不便携；夏天佩戴可能热；不适合运动。\\n    *   **适合场景**：长途旅行（飞机、高铁）、办公室、居家听音乐、专业音频工作。\\n\\n3.  **颈挂式蓝牙耳机**\\n    *   **形态**：两个耳机之间通过一根颈带（通常是柔性材质）连接。\\n    *   **优点**：不易丢失（挂在脖子上）；续航比真无线长；佩戴相对稳固。\\n    *   **缺点**：依然有线缆束缚；运动时颈带可能会有晃动。\\n    *   **适合场景**：对续航要求高、又担心丢失的用户，以及部分运动场景。\\n\\n---\\n\\n### 二、核心选购要点\\n\\n无论你选哪种类型，以下几点都是决定体验好坏的关键：\\n\\n#### 1. 音质\\n*   **蓝牙编码格式**：这是决定音质上限的关键。\\n    *   **SBC**：最基础的格式，所有蓝牙耳机都支持，音质一般。\\n    *   **AAC**：苹果设备的标配，音质优于SBC，安卓旗舰也普遍支

In [ ]:
# 中文
prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的公司的最佳名称是什么?"
)
chain = (prompt|llm)
product = "蓝牙耳机"
chain.invoke(product)

AIMessage(content='\n为一家制造蓝牙耳机的公司选择最佳名称，需要考虑品牌定位、目标受众和产品特性。一个好的名字应该易于记忆、发音，并能传达出产品的核心价值。\n\n以下是几个不同风格和方向的最佳名称建议，并附上解释，希望能给你带来灵感。\n\n---\n\n### 一、 科技导向型 (突出技术、连接与性能)\n\n这类名字适合强调音质、降噪技术或创新功能的品牌，目标客户是科技爱好者和对音质有高要求的用户。\n\n1.  **AuroraTech (极光科技)**\n    *   **含义**: Aurora (极光) 象征着绚丽、清晰和前沿的信号，Tech (科技) 直接点明行业属性。名字充满未来感和科技感。\n    *   **优点**: 听起来高端、专业，暗示着卓越的无线连接技术。\n\n2.  **CoreLink (核芯连接)**\n    *   **含义**: Core (核心) 代表着强大的处理芯片和音质核心，Link (连接) 强调了蓝牙的稳定与无缝。\n    *   **优点**: 简洁有力，传达出“核心技术、稳定连接”的可靠感。\n\n3.  **SignalFlow (信号流)**\n    *   **含义**: 描述了声音信号如流水般顺畅、无延迟地传输。\n    *   **优点**: 形象地表达了产品的核心优势——流畅的音频体验。\n\n4.  **Quantum Acoustics (量子声学)**\n    *   **含义**: Quantum (量子) 带有极强的科技前沿感，Acoustics (声学) 是音频领域的专业术语。\n    *   **优点**: 听起来非常专业、高端，适合定位为顶级音频专家的品牌。\n\n---\n\n### 二、 体验与情感导向型 (突出感受、自由与沉浸)\n\n这类名字着重于用户佩戴耳机时的感受，如沉浸感、自由感或放松感，适合生活方式类品牌。\n\n1.  **EchoZen (回响之禅)**\n    *   **含义**: Echo (回响) 与声音相关，Zen (禅) 代表着宁静、专注和内心平和。\n    *   **优点**: 特别适合主打降噪功能的耳机，营造出一种“隔绝尘嚣，静享音乐”的意境。\n\n2.  **SoundScape (音景)**\n   

In [15]:
aimsg = chain.invoke(product)

RateLimitError: Error code: 429 - {'error': {'code': '1113', 'message': '余额不足或无可用资源包,请充值。'}}

## 2. Sequential Chains

### 2.1 SimpleSequentialChain

顺序链（Sequential Chains）是按预定义顺序执行其链接的链。具体来说，我们将使用简单顺序链（SimpleSequentialChain），这是顺序链的最简单类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入

In [13]:
from langchain.chains import SimpleSequentialChain

In [14]:
# OPENAI_API_KEY = "********"      #"填入你的专属的API key"
llm = ChatOpenAI(temperature=0.9,openai_api_key=OPENAI_API_KEY)

子链 1

In [15]:

# 提示模板 1 ：这个提示将接受产品并返回最佳名称来描述该公司
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

子链 2

In [16]:

# 提示模板 2 ：接受公司名称，然后输出该公司的长为20个单词的描述
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

现在我们可以组合两个LLMChain，以便我们可以在一个步骤中创建公司名称和描述

In [17]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

给一个输入，然后运行上面的链

In [18]:
product = "Queen Size Sheet Set"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Linens.
Royal Linens is a luxury home textile company specializing in fine bedding, towels, and table linens for the discerning consumer.

> Finished chain.


'Royal Linens is a luxury home textile company specializing in fine bedding, towels, and table linens for the discerning consumer.'

In [21]:
# 中文
first_prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的公司的最佳名称是什么?"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(   
    "写一个20字的描述对于下面这个\
    公司：{company_name}的"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
product = "蓝牙耳机"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
蓝牙之音。
蓝牙之音，专业生产蓝牙音响的公司，提供高质量的无线音乐体验。

> Finished chain.


'蓝牙之音，专业生产蓝牙音响的公司，提供高质量的无线音乐体验。'

### 2.2 SequentialChain

当只有一个输入和一个输出时，简单的顺序链可以顺利完成。但是当有多个输入或多个输出时该如何实现呢？

我们可以使用普通的顺序链来实现这一点

In [22]:
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI    #导入OpenAI模型
from langchain.prompts import ChatPromptTemplate   #导入聊天提示模板
from langchain.chains import LLMChain    #导入LLM链。

初始化语言模型

In [23]:
# OPENAI_API_KEY = "********"      #"填入你的专属的API key"
llm = ChatOpenAI(temperature=0.9,openai_api_key=OPENAI_API_KEY)

接下来我们将创建一系列的链，然后一个接一个使用他们

In [24]:
#子链1

# prompt模板 1: 翻译成英语（把下面的review翻译成英语）
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: 输入：Review 输出： 英文的 Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [25]:
#子链2

# prompt模板 2: 用一句话总结下面的 review
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: 输入：英文的Review   输出：总结
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [26]:
#子链3

# prompt模板 3: 下面review使用的什么语言
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: 输入：Review  输出：语言
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [27]:
#子链4

# prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: 输入： 总结, 语言    输出： 后续回复
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [28]:
# 对四个子链进行组合

#输入：review    输出：英文review，总结，后续回复 
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

让我们选择一篇评论并通过整个链传递它，可以发现，原始review是法语，可以把英文review看做是一种翻译，接下来是根据英文review得到的总结，最后输出的是用法语原文进行的续写信息。

In [30]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds the taste of the product mediocre, and suspects it might be an old batch or counterfeit due to the foam not holding up.',
 'followup_message': "Réponse de suivi:\nJe vous remercie pour votre commentaire sur le goût du produit. Nous sommes désolés d'apprendre que cela n'a pas répondu à vos attentes. Nous prenons très au sérieux toute préoccupation concernant la qualité de nos produits et nous examinerons attentivement votre cas pour déterminer s'il s'agit d'un lot périmé ou contrefait. Nous apprécions votre feedback et nous nous efforcerons de faire en sorte que nos produits correspondent toujours à vos attente

## 3. Router Chain（路由链）

到目前为止，我们已经学习了LLM链和顺序链。但是，如果您想做一些更复杂的事情怎么办？

一个相当常见但基本的操作是根据输入将其路由到一条链，具体取决于该输入到底是什么。如果你有多个子链，每个子链都专门用于特定类型的输入，那么可以组成一个路由链，它首先决定将它传递给哪个子链，然后将它传递给那个链。

路由器由两个组件组成：

- 路由器链本身（负责选择要调用的下一个链）
- destination_chains：路由器链可以路由到的链

举一个具体的例子，让我们看一下我们在不同类型的链之间路由的地方，我们在这里有不同的prompt:  

### 定义提示模板

In [31]:
#第一个提示适合回答物理问题
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


#第二个提示适合回答数学问题
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""


#第三个适合回答历史问题
history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

首先需要定义这些提示模板，在我们拥有了这些提示模板后，可以为每个模板命名，然后提供描述。例如，第一个物理学的描述适合回答关于物理学的问题，这些信息将传递给路由链，然后由路由链决定何时使用此子链。

In [32]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

### 导入相关的包

In [33]:
from langchain.chains.router import MultiPromptChain  #导入多提示链
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

### 定义语言模型

In [34]:
# OPENAI_API_KEY = "********"      #"填入你的专属的API key"
llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)

### LLMRouterChain（此链使用 LLM 来确定如何路由事物）

在这里，我们需要一个**多提示链**。这是一种特定类型的链，用于在多个不同的提示模板之间进行路由。
但是，这只是你可以路由的一种类型。你也可以在任何类型的链之间进行路由。

这里我们要实现的几个类是LLM路由器链。这个类本身使用语言模型来在不同的子链之间进行路由。
这就是上面提供的描述和名称将被使用的地方。

#### 创建目标链  
目标链是由路由链调用的链，每个目标链都是一个语言模型链

In [36]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

#### 创建默认目标链
除了目标链之外，我们还需要一个默认目标链。这是一个当路由器无法决定使用哪个子链时调用的链。在上面的示例中，当输入问题与物理、数学、历史或计算机科学无关时，可能会调用它。

In [37]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

#### 创建LLM用于在不同链之间进行路由的模板
这包括要完成的任务的说明以及输出应该采用的特定格式。

In [38]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

#### 构建路由链
首先，我们通过格式化上面定义的目标创建完整的路由器模板。这个模板可以适用许多不同类型的目标。
因此，在这里，您可以添加一个不同的学科，如英语或拉丁语，而不仅仅是物理、数学、历史和计算机科学。

接下来，我们从这个模板创建提示模板

最后，通过传入llm和整个路由提示来创建路由链。需要注意的是这里有路由输出解析，这很重要，因为它将帮助这个链路决定在哪些子链路之间进行路由。

In [39]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

#### 最后，将所有内容整合在一起，创建整体链路

In [40]:
#多提示链
chain = MultiPromptChain(router_chain=router_chain,    #l路由链路
                         destination_chains=destination_chains,   #目标链路
                         default_chain=default_chain,      #默认链路
                         verbose=True   
                        )

#### 进行提问

如果我们问一个物理问题，我们希望看到他被路由到物理链路

In [41]:
# 问题：什么是黑体辐射？
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that falls on it and emits radiation at all wavelengths. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding the behavior of stars, as well as in the development of technologies such as incandescent light bulbs and infrared cameras."

In [44]:
# 问题：什么是黑体辐射？
chain.run("什么是热力学第二定率？")



> Entering new MultiPromptChain chain...
physics: {'input': '什么是热力学第二定率？'}
> Finished chain.


'热力学第二定律是指热量不会自发地从低温物体传递到高温物体，而只有在外界施加功的情况下，热量才会从低温物体传递到高温物体。此外，热力学第二定律还指出，任何一个封闭系统的熵都不会减少，而只会增加或保持不变。'

In [24]:
#中文
chain.run("黑洞是种什么物质?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is a black hole made of?'}
> Finished chain.


'A black hole is not made of any material substance that we know of. It is formed by the collapse of a massive object, such as a star, into an infinitely small point called a singularity. This singularity has infinite density and gravity, and is surrounded by an event horizon, which is the point of no return for anything that gets too close to the black hole. So, in essence, a black hole is made of nothing but gravity.'

如果我们问一个数学问题，我们希望看到他被路由到数学链路

In [22]:
# 问题：2+2等于多少？
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'As an AI language model, I can answer this question. The answer to 2 + 2 is 4.'

如果我们传递一个与任何子链路都无关的问题时，会发生什么呢？

这里，我们问了一个关于生物学的问题，我们可以看到它选择的链路是无。这意味着它将被**传递到默认链路，它本身只是对语言模型的通用调用**。语言模型幸运地对生物学知道很多，所以它可以帮助我们。

In [40]:
# 问题：为什么我们身体里的每个细胞都包含DNA？
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of each cell. DNA contains the instructions for the synthesis of proteins, which are essential for the structure and function of cells. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next. Therefore, every cell in our body needs DNA to carry out its specific functions and to maintain the integrity of the organism as a whole.'

In [45]:
# 中文
chain.run("你知道李白是谁嘛?")



> Entering new MultiPromptChain chain...
History: {'input': '你知道李白是谁嘛?'}
> Finished chain.


'李白是唐朝时期的一位著名诗人。他的诗歌以豪放、奔放、自由的风格著称，被誉为“诗仙”。他的作品涉及广泛，包括山水田园、历史传说、哲理思考等多个方面，对中国古典文学的发展产生了深远的影响。'